In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data", one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


## What we will be doing

- Write a few helper functions (init_weights, init_bias, conv2d, max pool, conv-layer)
- Variables, placeholders, cost, optimizers
- Train CNN


In [2]:
def init_wgts(shape):
    init_random_dist = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(init_random_dist)

def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape = shape)
    return tf.Variable(init_bias_vals)

In [3]:
def conv2d(x, W):
    """ x -> [batch,H,W,Channels]
    W -> [filter H, Filter W, Channel IN, Channel OUT]
    Use a stride of 1, keep padding same"""
    
    return tf.nn.conv2d(x, W, strides = [1,1,1,1], padding = 'SAME')

In [4]:
def max_pool_2by2(x):
    """ x -> [batch,H,W,Channels]"""
    return tf.nn.max_pool(x, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')

In [5]:
def convolutional_layer(input_x, shape):
    W = init_wgts(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x, W) + b)

In [6]:
def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_wgts([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b

In [7]:
# Placeholders
x = tf.placeholder(tf.float32, shape = [None, 784])
y = tf.placeholder(tf.float32, shape = [None, 10])

In [8]:
x_image = tf.reshape(x,[-1, 28, 28, 1])

In [9]:
# Layer 1
convo_1 = convolutional_layer(x_image, shape = [5, 5, 1, 32])
convo_1_pooling = max_pool_2by2(convo_1)

# Layer 2
convo_2 = convolutional_layer(convo_1_pooling, shape = [5, 5, 32, 64])
convo_2_pooling = max_pool_2by2(convo_2)

# Flatten it out
convo_2_flat = tf.reshape(convo_2_pooling, [-1, 7*7*64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat, 1064))

In [10]:
# Dropout
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one, keep_prob = hold_prob)

In [11]:
# Y-pred label
y_pred = normal_full_layer(full_one_dropout, 10)

In [12]:
# loss
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = y, logits = y_pred))

In [13]:
# Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate = 0.0001)
train = optimizer.minimize(cross_entropy)

In [14]:
# Initialize
init = tf.global_variables_initializer()

In [15]:
steps = 5000

In [16]:
session = tf.Session()

with session as sess:
    sess.run(init)
    
    for i in range(steps):
        batch_x, batch_y = mnist.train.next_batch(500)
        #grab from next batch
        sess.run(train, feed_dict = {x:batch_x, y:batch_y, hold_prob:0.5})
        #print out message every 100 steps
        if i % 100 == 0:
            print("Currently on step {}".format(i))
            print("Accuracy is: ")
            # Test the train model
            matches = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y, 1))
            acc = tf.reduce_mean(tf.cast(matches, tf.float32))
            print(sess.run(acc, feed_dict = {x:mnist.test.images, y:mnist.test.labels, hold_prob: 1.0}))
            print('\n')

Currently on step 0
Accuracy is: 
0.0654


Currently on step 100
Accuracy is: 
0.9195


Currently on step 200
Accuracy is: 
0.9478


Currently on step 300
Accuracy is: 
0.9601


Currently on step 400
Accuracy is: 
0.9657


Currently on step 500
Accuracy is: 
0.9696


Currently on step 600
Accuracy is: 
0.9742


Currently on step 700
Accuracy is: 
0.9765


Currently on step 800
Accuracy is: 
0.9774


Currently on step 900
Accuracy is: 
0.9799


Currently on step 1000
Accuracy is: 
0.9814


Currently on step 1100
Accuracy is: 
0.9824


Currently on step 1200
Accuracy is: 
0.9847


Currently on step 1300
Accuracy is: 
0.9846


Currently on step 1400
Accuracy is: 
0.9849


Currently on step 1500
Accuracy is: 
0.9862


Currently on step 1600
Accuracy is: 
0.9865


Currently on step 1700
Accuracy is: 
0.9873


Currently on step 1800
Accuracy is: 
0.9878


Currently on step 1900
Accuracy is: 
0.9886


Currently on step 2000
Accuracy is: 
0.9882


Currently on step 2100
Accuracy is: 
0.9884


